# funcard Program

In [ ]:
import smartleia as sl

In [ ]:
reader = sl.LEIA()

In [ ]:
print('Connection with', reader.ser.name)

In [ ]:
reader.flasher()

In bash
``` bash
avrdude -p m8515 -c avrisp2 -P COM7 -U flash:w:`ls smartleia-target-funcard/src/build/aes*.hex`:i -U eeprom:w:`ls smartleia-target-funcard/src/build/eedata*.hex`:i
```

Or, in cmd
```
avrdude -p m8515 -c avrisp2 -P COM7 -U flash:w:smartleia-target-funcard\src\build\aes-300721-124817.hex:i -U eeprom:w:smartleia-target-funcard\src\build\eedata-300721-124818.hex:i
```

# Communication with card

Example of communication with korean financial IC card

```
ATR(Cold): 3B6800000073C84011009000

[APDU COMMAND]: 00A4040007 D4106509900010
[RECEIVE DATA]: 612A

[APDU COMMAND]: 00C000002A
[RECEIVE DATA]: 6F288407D4106509900010A51D5010C7CFB3AAC7F6B1DDC4ABB5E500000000BF0C0801010000000000819000

[APDU COMMAND]: 90E4008120 0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF
[RECEIVE DATA]: 6120

[APDU COMMAND]: 00C0000020
[RECEIVE DATA]: AD5F4EADBE5362BAF89369EC997D9EAFA275D41C3DDFB009ED1506B72A1390009000
```

In [ ]:
import smartleia as sl

In [ ]:
reader = sl.LEIA()
print('Connection with', reader.ser.name)

In [ ]:
reader.is_card_inserted()

### TODO: try negotiation

``` python
reader.configure_smartcard(negotiate_pts=False)
reader.configure_smartcard(protocol_to_use=1,  # Use T=1
                           ETU_to_use=None,    # Let the reader determine the ETU to use
                           freq_to_use=None,   # Let the reader determine the freq to use
                           negotiate_pts=True, # Let the reader negotiate the PTS
                           negotiate_baudrate=True
)
reader.configure_smartcard(protocol_to_use=1)
```

In [ ]:
reader.set_trigger_strategy(0,
                            [sl.TriggerPoints.TRIG_GET_ATR_PRE,
                             sl.TriggerPoints.TRIG_GET_ATR_POST,
                             sl.TriggerPoints.TRIG_POST_RESP],
                            delay=0,
                            single=0)
# TRIG_PRE_SEND_APDU
# TRIG_POST_RESP 

In [ ]:
reader.set_trigger_strategy(0,
                            [sl.TriggerPoints.TRIG_GET_ATR_PRE,
                             sl.TriggerPoints.TRIG_GET_ATR_POST],
                            delay=0,
                            single=0)
# TRIG_PRE_SEND_APDU
# TRIG_POST_RESP 

In [ ]:
ATR = reader.get_ATR() 
print(ATR)
print(f"We are using protocol T={ATR.T_protocol_curr} and the frequence of the ISO7816 clock is {ATR.f_max_curr/1000} kHz !")

In [ ]:
reader.configure_smartcard(protocol_to_use=0)

In [ ]:
ATR = reader.get_ATR() 
print(ATR)
print(f"We are using protocol T={ATR.T_protocol_curr} and the frequence of the ISO7816 clock is {ATR.f_max_curr/1000} kHz !")

In [ ]:
reader.reset()

In [ ]:
import binascii

#apdu_ = sl.create_APDU_from_bytes(binascii.unhexlify(''))
#apdu_ = sl.APDU(cla=0x00, ins=0x01, p1=0x00, p2=0x00)
# cla ins p1 p2 lc data le

apdu_selectdf = sl.create_APDU_from_bytes(binascii.unhexlify('00A4040007D4106509900010'))
apdu_getresponse_selectdf = sl.create_APDU_from_bytes(binascii.unhexlify('00C000002A'))
apdu_getcipher = sl.create_APDU_from_bytes(binascii.unhexlify('90E4008120'
                                                              + '0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF'))
apdu_getresponse_getcipher = sl.create_APDU_from_bytes(binascii.unhexlify('00C0000020'))

In [ ]:
resp = reader.send_APDU(apdu_selectdf)
print(resp)
print(resp.sw2)

In [ ]:
resp = reader.send_APDU(apdu_getresponse_selectdf)
print(resp)
print('Response: ', ''.join(['%02X' % i for i in resp.data]))

In [ ]:
print(apdu_getcipher)
resp = reader.send_APDU(apdu_getcipher)
print(resp)

In [ ]:
resp = reader.send_APDU(apdu_getresponse_getcipher)
print(resp)
print('Response: ', ''.join(['%02X' % i for i in resp.data]))

In [ ]:
reader.close()

아래 오실로스코프 파형 불러오기는 https://github.com/mcsmonk/dsopy-lecroy 코드 이용

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lecroywin32com import LeCroyScope

In [ ]:
ip = '163.152.90.119'

In [ ]:
dso = LeCroyScope(ip)
dso.clear()

In [ ]:
desc = dso.get_wavedesc()
print(desc)

In [ ]:
t2 = dso.get_waveform('C1')

In [ ]:
plt.plot(t2)
plt.show()